In [15]:
import cv2
import json
import os

import numpy as np
import tensorflow.keras.applications.nasnet as nasnet

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.nasnet import preprocess_input
from data_preprocessing import DataPreprocessing

def extract_frame_features(frame):
    img = image.img_to_array(frame)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    base_model = nasnet.NASNetLarge(weights='imagenet')
    features = base_model.predict(img)

    return features.flatten()


def preprocess_data(input_json_file, output_path, frame_width, frame_height):
    
    output_dir = os.path.join(output_path)
    os.makedirs(output_dir, exist_ok=True)

    dp = DataPreprocessing()

    # Read the JSON file
    with open(input_json_file, 'r') as json_file:
        data = json.load(json_file)

    for file in data:
        video_path = file['SENTENCE_FILE_PATH']
        
        video_name = os.path.split(video_path)[-1]
        frame_count = 0

        # Open the input video
        cap = cv2.VideoCapture(video_path)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # cropped_frame = dp.crop(frame, 300, 0, 680, 720)
            # resized_frame = dp.resize(cropped_frame, frame_width, frame_height)
            # noise_reduced_frame = dp.reduce_noise(resized_frame, 10, 7, 21)
            # noise_added_frame = dp.add_noise(noise_reduced_frame, 'poisson', 0, 1)
            # transformed_frame = dp.spatial_transformation(noise_added_frame, 0, True, False)
            color_jittered_frame = dp.color_jitter(frame, 0, 0, 0, 60)

            # features = extract_frame_features(resized_frame)
            # video_features.append(features)
            # video_features = np.stack(video_features)
            # print(video_features.shape)

            # Write the resized frame to the output video
            frame_name = f"{video_name}_frame{frame_count}.jpg"
            cv2.imwrite(os.path.join(output_dir, frame_name), color_jittered_frame)

            frame_count += 1

        # Release the video capture and writer objects
        cap.release()
        break


input_json_file = '../Dataset/test.json'
output_path = '../Dataset/transformed/'
preprocess_data(input_json_file, output_path, 331, 331)


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<3, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 5, -1>, cv::impl::(anonymous namespace)::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<3, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 5, -1>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
